In [8]:
%%writefile LaminarFlow/LaminarFlow.py
import tensorflow as tf
import pickle as pkl
    
class _tf_temp():
    def __init__(self, name):
        self.name = name

class CruiseControl():
    """
    Laminar Flow's Cruise Control method automates several
    time saving tasks for Tensorflow, allowing for quicker
    prototyping and testing.
    
    Among these abilities is automatic saving in an encapsulated
    tensorflow session. This makes it so you don't need to
    keep open a session to keep the values. However, it does
    require you to use `tf.get_variable` to define variables
    instead of `tf.Variable` directly.
    
    On top of that, automatic initialization of uninitialized
    variables allows for this structure to be dynamically updated
    and have no cat and mouse hunt for errors.
    
    Pickling and unpickling has been implemented in this class
    under the strict condition that all non-variable tensors
    used in args to the `add` function are the result of previous
    functions input to the `add` function. That is, when calling
    `add`, only use direct attributes of that CruiseControl
    instance or variables controlled by it.
    
    More documentation to follow.
    """
#Constructor
    def __init__(self, save_file_name, unique_identifier = None):
        #collect_variables()
        self._vars = set()
        self._initialized = set()
        self._var_pkl = list()
        self._file_name = save_file_name
        self.saver = None
        self._uuid = unique_identifier if unique_identifier else hex(id(self))[2:]
        try:
            with tf.variable_scope(self._uuid):
                with tf.variable_scope(self._uuid):
                    tf.get_variable("initialized",shape=[1])
        except:
            raise NameError("Error: {} is an invalid unique identifier.".format(self._uuid))
#Structure
    def add(self, name, function, *args, **kwargs):
        if hasattr(self, name):
            raise AttributeError("Error: {} already defined.".format(name))
        current = set(tf.all_variables())
        #sanitize
        sanitized_args = []
        start = len(self._uuid)
        for i in args:
            try:
                pkl.dumps(i)
                sanitized_args.append(i)
            except:
                restart = i.name[start:].find('/') + start + 1
                sanitized_args.append(_tf_temp(i.name[restart:i.name.rfind(":")]))
        #Test keyword args.
        try:
            pkl.dumps(kwargs)
        except:
            raise ValueError("Error: Unable to handle tf objects in keyword args currently.")
        with tf.variable_scope(self._uuid):
            #unsanitize
            unsanitized_args = []
            for i in args:
                if isinstance(i, _tf_temp):
                    try:
                        unsanitized_args.append(tf.get_variable(i.name))
                    except:
                        end = i.name.find('/')
                        unsanitized_args.append(getattr(self, i.name[:end]))
                else:
                    unsanitized_args.append(i)
            with tf.variable_scope(name):
                setattr(self, name, function(*unsanitized_args, **kwargs))
        self._vars |= set(tf.all_variables()) - current
        self._var_pkl.append([name, function, sanitized_args, kwargs])
        return self
#Features
    def setFile(self, save_file_name):
        self._file_name = save_file_name
#Values
    def save(self, save_file_name = None):
        variables = []
        start = len(self._uuid)
        for i in self._vars:
            restart = i.name[start:].find('/') + start + 1
            if tf.is_variable_initialized(i).eval():
                try:
                    variables.append((i.name[restart:i.name.rfind(":")],i.value().eval()))
                except:
                    #TODO: Don't do this.
                    pass
        with open(self._file_name, "wb") as file:
            pkl.dump(variables, file)
    def load(self, save_file_name = None):
        try:
            with open(self._file_name, "rb") as file:
                variables = pkl.load(file)
            with tf.variable_scope(self._uuid, reuse=True):
                for i in variables:
                    tf.get_variable(i[0]).assign(i[1]).eval(session=self._sess)
        except:
            pass
#Serialization
    def __reduce__(self):
        return (CruiseControl, (self._file_name,), self._var_pkl)
    def __setstate__(self, state):
        for i in state:
            self.add(i[0],i[1],*i[2],**i[3])
        return self
#Functionality
    def __enter__(self):
        self._sess = tf.Session()
        uninitialized = self._vars - self._initialized
        # Fixed in TF 0.9. Upgrade now!
        #uninitialized = tf.report_uninitialized_variables(self._vars).eval(session=self._sess)
        self._sess.run(tf.initialize_variables(list(uninitialized)))
        self.load()
        self._initialized = set(x for x in self._vars)
        return self._sess.__enter__()
    def __exit__(self, *args):
        self.save()
        return self._sess.__exit__(*args)

Overwriting LaminarFlow/LaminarFlow.py


In [2]:
import tensorflow as tf

def clu(x, outdim, indim=None):
    if indim:
        try:
            indim = list(indim)
        except:
            indim = [indim]
    else:
        indim = x.get_shape()[1:].as_list()
    try:
        outdim = list(outdim)
    except:
        outdim = [outdim]
    C = tf.get_variable("C", initializer=tf.truncated_normal(indim+outdim, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.truncated_normal(outdim, stddev=0.1))
    cluKernel = -tf.reduce_sum(tf.abs(tf.sub(tf.expand_dims(x,len(indim)+1),
                                        tf.expand_dims(C,0))),
                          1) + b
    cluKernel._C = C
    cluKernel._b = b
    return cluKernel

def lin(x, shape):
    W = tf.get_variable("W", initializer=tf.truncated_normal(shape, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.truncated_normal(shape[1:], stddev=0.1))
    linKernel = tf.matmul(x,W) + b
    linKernel._W = W
    linKernel._b = b
    return linKernel

#Convolution and Maxpool shortcuts
def conv2d(x, shape):
    W = tf.get_variable("W", initializer=tf.truncated_normal(shape, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=shape[-1:]))
    convKernel = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') + b
    convKernel._W = W
    convKernel._b = b
    return convKernel

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def cross_entropy(y,yhat):
    return -tf.reduce_sum(y*tf.log(yhat + 1e-9)) #Without epsilon, it crashes.

def accuracy_test(y,yhat):
    correct_prediction = tf.equal(tf.argmax(yhat,1), tf.argmax(y,1))
    return tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [3]:
from LaminarFlow import CruiseControl

test = CruiseControl("test")
test.add('input',tf.placeholder, "float", shape=[None, 784])
#test.add('expected_output', tf.placeholder, "float", shape=[None, 10])
test.add('h', lin, test.input,[784,10])
test.add('output', tf.nn.softmax, test.h)
#test.add('optim', tf.train.AdadeltaOptimizer, 1e-4)
#test.add('loss', cross_entropy, test.expected_output, test.output)
#test.add('train_step', test.optim.minimize, test.loss)

In [4]:
import numpy as np

with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))

[[ 0.00099203  0.20124029  0.01566002  0.00757608  0.06891601  0.00304694
   0.00833866  0.68667036  0.00223492  0.00532476]]
[[ 0.00099203  0.20124029  0.01566002  0.00757608  0.06891601  0.00304694
   0.00833866  0.68667036  0.00223492  0.00532476]]


In [5]:
import pickle as pkl

test2 = pkl.loads(pkl.dumps(test))
with test2 as sess:
    print(test2.output.eval({test2.input:np.ones([1,784])}))

[[ 0.00099203  0.20124029  0.01566002  0.00757608  0.06891601  0.00304694
   0.00833866  0.68667036  0.00223492  0.00532476]]


In [6]:
test = CruiseControl("test")
test.add('input',tf.placeholder, "float", shape=[None, 784])
test.add('expected_output', tf.placeholder, "float", shape=[None, 10])
test.add('h', lin, test.input,[784,10])
test.add('output', tf.nn.softmax, test.h)
test.add('optim', tf.train.AdadeltaOptimizer, 1e-4)
test.add('loss', cross_entropy, test.expected_output, test.output)
test.add('train_step', test.optim.minimize, test.loss)
#testing
test._initialized = set(x for x in test._vars)
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))

[[ 0.00099203  0.20124029  0.01566002  0.00757608  0.06891601  0.00304694
   0.00833866  0.68667036  0.00223492  0.00532476]]


In [7]:
test.add('relu', tf.nn.relu, test.h)
test.add('h2', lin, test.relu, [10,10])
test.add('output2', tf.nn.softmax, test.h2)
test.add('loss2', cross_entropy, test.expected_output, test.output2)
test.add('train_step2', test.optim.minimize, test.loss2)
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output2.eval({test.input:np.ones([1,784])}))

[[ 0.00099203  0.20124029  0.01566002  0.00757608  0.06891601  0.00304694
   0.00833866  0.68667036  0.00223492  0.00532476]]


FailedPreconditionError: Attempting to use uninitialized value 7f1afc141588/h/W
	 [[Node: 7f1afc141588/h/W/read = Identity[T=DT_FLOAT, _class=["loc:@7f1afc141588/h/W"], _device="/job:localhost/replica:0/task:0/gpu:0"](7f1afc141588/h/W)]]
Caused by op '7f1afc141588/h/W/read', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 589, in launch_instance
    app.start()
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.4/dist-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/py

In [43]:
tf.is_variable_initialized(t).eval()

True